# HW4 LSH 
(執行約20分鐘)

# Initial and setting config

In [1]:
#initial
from pyspark import SparkContext, SparkConf
import re
import string
import time
sc.stop()
conf = SparkConf().setMaster("local").setAppName("LSH")
conf = SparkConf().set("spark.default.parallelism", 1)
sc = SparkContext(conf = conf)

# Parsing and Constructing Shingles 
1. Using sc.wholeTextFiles() to get 101 datas first
2. Uisng map function parsing(x) to parsing the lines to the Words of each data RDD
3. After parsing lines, we will using map function shingle_3(x) to output the 3-shingles of each Set of Words in data
4. After step 3, flatMap first with calling distinct() to get all possible shingles, and then label them with index(i.e. row th) by calling zipWithIndex()

## parsing(x):
1. It's a map function for parsing the words in documents by splitting:
#####  space' '   
##### period'.' 
##### end_line'\n' 
##### punctuation ','
##### 雙引號'"'
##### 
2. Note that "".join means transfer the list to string, since we will use the string attribution 'replace()'
3. Also, I use replace instead of split since I splitting not only by one 標點符號, so I first repalce them to ''

## shingle_3(x):
1. It's a map function for genarating all possible 3-shingle by scaning every 連續的 3 words iteratively
2. The iteration range should be len(line_list)-2, since we should finish at last 3th word.


In [2]:
#get input data => RDD
def parsing(x):
    while(x.find('.') != -1):
        a = x.find('.')
        l = list(x)
        l[a] = ''
        x = "".join(l)
    x = x.replace(',','')
    x = x.replace('"','')
    x = x.replace('\n',' ')
    return x
def shingle_3(x):
    line = x.split(' ')
    line_list = list(line)
    while('' in line_list):
        line_list.remove('')
    result = []
    for i in range(len(line_list)-2):
        s = line_list[i]+' '+line_list[i+1]+' '+line_list[i+2]
        result.append(s)
    return result

shingle_table = sc.wholeTextFiles("athletics/").map(lambda x:x[1]).map(parsing).filter(lambda x:x!='').map(shingle_3).flatMap(lambda x:x).distinct()
label_shingle_table = shingle_table.map(lambda x:("data",x)).zipWithIndex().map(lambda x:(x[0][0],(x[0][1],x[1])))

# Construct Shingles/Documents Table
1. To determine whether the shingle is in a data, we can iteratively scan every data.
2. In each iteration, we first use sc.textFile() to open a data to RDD, and calling map function parsing2(x) to find its all 3-shingle, after that we will get a RDD with several <key,value> the key is the number of document the value is several shingles, so in order to make them to a RDD with only one data, we merge them by calling reduce function reduce_add_with_space(x,y). 
3. Join the RDD from step2 with the label_shingle_table as shingle_reference which we have generate before.
4. Calling map function detect(x) to generate a col of Shingles/Documents Table.
5. Once we generate a new col, we should union it with previous cols before we enter next iteration.
6. After all iterations, we will get a RDD which <key,values> is 
#### <number_of_document(i.e. col), vector of that col>.

## parsing2(x)
1. it do the same thing as parsing(x), but we need to label the key "data" as join-key with label_shingle_table.

## detect(x)
1. It's a map function which do the same first as shingle(x), to generate possible 3-shingle for each data.
2. But simultaneously, we should detect the shingle from data is same as each shingle in shingle_table, if true set 1 to that (shingle,document) index, else set 0.
3. after this fuction we can generate a columne of Shingles/Documents Table.

## reduce_add_with_space(x,y)
1. It's the reduce function to let us merge the values(contain with several shingles) together and split each shingle with space '_' 

In [3]:
def get_num(num):
    if num<10:return"00"+str(num)
    elif 10<=num<100: return "0"+str(num)
    else: return str(num)
def parsing2(x):
    x = x.replace('.','')
    x = x.replace(',','')
    x = x.replace('\n',' ')
    x = x.replace('"','')
    x = x.replace('  ',' ')
    return "data",x
def reduce_add_with_space(x,y):
    return x+' '+y

def detect(x):
    line = x[1][1].split(' ')
    line_list = list(line)
    for i in range(len(line_list)-2):
        s = line_list[i]+' '+line_list[i+1]+' '+line_list[i+2]
        if s == x[1][0][0]:
            return "data",(x[1][0][1],1)
    return "data",(x[1][0][1],0)
def split_data(x):
    result = []
    for i in x[0]:
        result.append(i)
    result.append(x[1])
    return result
cols = None
cols_2 = None
next_cols = None

tStart = time.time()
for i in range(101):
    old_cols = cols
    #print("iteration"+str(i))
    doc_num = get_num(i+1)
    data = sc.textFile("athletics/"+doc_num+".txt").map(parsing2).filter(lambda x:x[1]!='').reduceByKey(reduce_add_with_space)
    join_set = label_shingle_table.join(data)
    next_col = join_set.map(lambda x:detect(x)).groupByKey()
    if i==0: cols = next_col
    else:
        cols = cols.union(next_col)
cols = cols.zipWithIndex().map(lambda x:(x[1]+1,list(x[0][1])))

# Min-Hashing
1. For each col in cols RDD, we calculate its hash values and find the min-hash value of it by calling a map function hashing(x)
2. After, hashing(x), we gain the signature matrix which store at RDD for line per columne

## hashing(x)
1. Before conducting this function,I make 100 different hash function first
2. the hash functions follow the formula (a*x+b)%p, where 'a' is a 100-size-list value random range form 1 to 101 and corresponding 100-size-list 'b' is random value range from 0 to 100, and then p is a prime number which is larger than numbers of all shingles, in 101 datas we find 26679 differents shingles, so we can set prime number p as 26681)
3. after pre-setting, we can iteratively travel the col vector in RDD(line per col), getting hash-value for different 100 hash function and find their min_hash value for each of them(hash_functions) in each col(i.e. document) 
4. Note that in the function, the first layer of iteration (i.e. for i in range(100):) means 100 differents hash function we have make, the second layer is to scanning the col in RDD.


In [4]:
#construct hash_function
import random
a = random.sample(range(1, 101), 100)
b = random.sample(range(0, 100), 100)
p = 26681

import math
def hashing(x):
    result = []
    col = x[0]
    for i in range(100):
        min_ = math.inf
        for j in x[1]:
            ans = math.inf
            if j[1]==1: 
                ans = (a[i]*(int(j[0])+1)+b[i])%p
            else: ans = math.inf
            if min_>ans:
                min_ = ans
                j_t = j
                ii = i
            elif (min_==ans) and (ans!=math.inf):return "Wrong",ans,j,p,j_t,i,ii,a[i],b[i] #for debug
        pair = (("h"+str(i+1)),min_)
        #if min_==0: pair = (("h"+str(i+1)),min_,a[i],b[i],j_t)
        result.append(pair)
    return x[0],result
#cols = cols.cache()
hashed_cols = cols.map(hashing)

# Locality-Sensitive Hashing - pre_setting
1. In homework, we set band b = 50 and band per rows r = 2 
2. So we should first label the col for every 2 row with a band number by using a map function label_band(x)
3. After band_labeling, we should re_construct the document-key RDD to the new RDD with band_num as key
4. To acheive it we can using flatMap() first to seperate every single element and then group them with same band number.

### label_band(x)
1. It's is a map function which let us mark every 2 elements in col vector with a same band number by using itration 

In [5]:
def label_band(x):
    result = []
    for i in range(50):
        band = i+1
        for j in range(2):
            pair = (str(band),("data"+str(x[0]),x[1][i*2+j]))
            result.append(pair)
    return result
label_band_num = hashed_cols.map(label_band)
final_label_band_num = label_band_num.flatMap(lambda x:x).groupByKey().map(lambda x:(x[0],list(x[1])))

# Locality-Sensitive Hashing - hash_to_buckets
1. Calling a map function hash_to_buckets(x) to hash the value to a buckets by the formula x%k, where k is buckets number(we set 1000)
2. After hashing, analysising the hashed elements inside which buckets in each Band, by calling map function get_hashing_result(x)

### hash_to_buckets(x)
1. It's a map function to calculate the hashing value (i.e. hash to a bucket) 

### get_hashing_result(x)
1. It's a map function which calculate the cumulation inside each bucket in each band

In [6]:
#hash to k buckets
k = 1000
def hash_to_buckets(x):
    list_ = []
    for i in x[1]:
        data_name = i[0]
        hash_func = i[1][0]
        bucket = i[1][1]%k
        pair = (bucket,(data_name,hash_func,i[1][1]))
        list_.append(pair)
    return x[0],list_
def get_hashing_result(x):
    buckets = [None]*k
    for i in range(k):
        list_ = []
        list_.append("B"+str(i))
        buckets[i] = list_
    for i in x[1]:
        j = i[0]
        try: 
            buckets[j].index(i[1][0])
        except:
            buckets[j].append(i[1][0])
    
    return x[0],buckets
    
after_hash_to_buckets = final_label_band_num.map(hash_to_buckets)
after_hash_to_buckets_final = after_hash_to_buckets.map(get_hashing_result)

# Locality-Sensitive Hashing - find_candidates
1. After hashing to bucket and get each bucket information in each band, we now want to find all the candidate pairs 
2. To find them, we can first find every data combinations in each bucket of each band by calling find_candicate_pairs(x)
3. After that, we will get the RDD which <key,values> is pairs of datas, and since different band may give us same candidates pairs, so we can use flatMap() plus distinct() to find unique candidate pairs.

### find_candicate_pairs(x):
1. It's a map function which use iteration to find out every pair in each bucke if that bucket contain two or more data.
2. return the all candicate pairs in the band.

In [7]:
import itertools
def find_candicate_pairs(x):
    candicate_pairs = []
    for i in x[1]:
        j = i[1:]
        size = len(j)
        if size>1:
            pairs = list(itertools.combinations(j, 2))
            candicate_pairs.append(pairs)
    return candicate_pairs
candicate_pairs = after_hash_to_buckets_final.map(find_candicate_pairs).flatMap(lambda x:x).flatMap(lambda x:x).map(lambda x:(int(re.sub("data",'',x[0])),int(re.sub("data",'',x[1])))).distinct()

# Locality-Sensitive Hashing - calculate_similarity
1. After getting all unique candidate pairs, we need to calculate their Jaccard_similarity
2. So first of all, we should give each pair their documents information(i.e. corresponding col vector) by label each pair with number of pair first and then use flatMap()to seperate them, and finally using join to append each of their required documnents information, and finally, since we have label their pair number beforehand, so we can easily group back them to pairs by calling reduce function cal_similarity(x,y)
3. Also when calling cal_similarity(x,y), we can simultaneously calculate their jaccard_similarity, the formula is:
### |x inetraction y|/|x union y|
#### 
4. After getting all the similarities of all candidate pairs, we sort the RDD decreasingly.

### label_pair(x)
1. It's a map function to make me easy to debug by the pair name

### delete_the_row_th(x):
1. It's a map function which let us delete the row_name of the elements inside the col vector RDD means:
#### (document_number,  [(0,1), (1,0), (2,1), (3,0), (4,1)....])  => (document_number, [1, 0, 1, 0, 1, ...])

### change_key(x)
1. It's a map function which help us change the key from document_number to pair_number after we have finishedd the join
2. So that we can group back the original pair and then calculate similarity by calling cal_similarity

### cal_similarity(x,y)
1. It's a reduce function which help us find original pair first and then calculate the pair's jaccard_similarity
2. using iteration to scan same index of two documents (cols vector) value and calculate the |x inetraction y| and |x union y|
3. |x inetraction y|+= 1 if both two documents col_vector[index] value equal to 1 (means both of them have same shingle)
4. |x union y| += 1 if either of them col_vector[index] equal to 1 (means either of them have a shingle which exist in shingles table)
5. after result of step 3 and 4, we can easily calculate the similarity = |x inetraction y| and |x union y|


In [8]:
def label_pair(x):
    p1 = (x[0][0],"pair"+str(x[1]))
    p2 = (x[0][1],"pair"+str(x[1]))
    return p1,p2
def delete_the_row_th(x):
    data = x[0]
    result = []
    for i in x[1]:
        result.append(i[1])
    return data,result
def change_key(x):
    key = x[1][0]
    data = x[0]
    return key,(x[0],x[1][1])
def cal_similarity(x,y):
    p1 = x[0]
    p2 = y[0]
    union = 0
    inter = 0
    for i in range(len(x[1])):
        relation = x[1][i]+y[1][i] 
        if relation==1: #union
            union+=1
        elif relation==2: #inter
            union+=1
            inter+=1
    similarity_jaccacrd = inter/union
    pair = None
    if p1<p2: pair = (p1,p2)
    elif p1>p2: pair = (p2,p1)
    return pair,similarity_jaccacrd

cols_new = cols.map(delete_the_row_th)

candicate_pairs_with_label = candicate_pairs.zipWithIndex().map(label_pair).flatMap(lambda x:x)
##join pair with its data 1s
candicate_with_data = candicate_pairs_with_label.join(cols_new)
#calculate sim
candicate_with_data_new = candicate_with_data.map(change_key)
Final = candicate_with_data_new.reduceByKey(cal_similarity).map(lambda x:x[1]).sortBy(lambda x:x[1],False)
#Final.take(10)

# Output top 10 pairs 
1. Collect the sorted RDD(Final) by calling take(10) to get to 10 similarity pairs 
2. And then use iteration to output the result as the format: (document1, document2 : similarity)
3. 結果以一般小數點表示(四捨五入至小數點第三位)4

In [19]:
for i in Final.take(10):
    pair = str(i[0])
    valid_f = '{:.3}'.format(i[1])
    sim = str(valid_f)
    print(pair+" : "+sim)

(12, 20) : 1.0
(52, 84) : 1.0
(47, 49) : 0.756
(30, 35) : 0.707
(49, 88) : 0.509
(48, 49) : 0.484
(23, 38) : 0.482
(14, 40) : 0.401
(47, 88) : 0.385
(47, 48) : 0.366
